# Basic assurance testing for ppxf
---
**Important**: remember to manually set the random seed each time to avoid differences in `ppxf` output due to noise!

* **Accurately recovering the smoothness of the SFH**: try SFHs with varying degrees of smoothness. Using the local minimum approach to find the optimal value for regul, does ppxf accurately return the degree of smoothness?
	* *Testing plan*: 
    	1. Single population w/ different smoothings. 1 young, then 1 old. What is the minimum smoothness in the SFH that ppxf can accurately recover? Find this value, and use this for all future tests.
        2. Multiple populations: how does ppxf fare when there are different smoothnesses in the different components? Try using a narrow width for the young stellar population, and a broad width for the old one. 
    	* make 2x "base" SFHs - one with only an old population, and one with a young & old component. Run ppxf with 4 different degrees of smoothness - with starburst durations of $10^6$, $10^7$ and $10^8$ yr (where we set the duration to the FWHM). See what happens.
    * Based on this result, adopt a standard starburst duration (adopt whatever value is able to be accurately recovered by ppxf) to use for future experiments.
* **Sensitivity to young stellar populations**: given a young stellar population superimposed on top of an older population, what is the minimum mass fraction of the young population for which ppxf can accurately return the SFH?
* **Effect of emission lines**: does the inclusion of emission lines affect the derived SFH at all? What about very broad emission lines?
* **Reducing the number of templates**: by how much can we degrade the temporal & metallicity sampling of the template grid & still accurately capture the SFH? (i.e., see if the fitting process can be sped up by using fewer templates)
* **Limiting the wavelength range to save time**: what happens if we use only the blue half of the spectrum?
* **Template mismatch**: how accurately can the SFH be recovered when the Geneva isochrones are used to generate the mock spectra, but the Padova isochrones are used in the fitting process?

### Text for paper

Accurately characterising the smoothness of the SFH is key to our work, as we need to determine when the most recent starburst activity began.
In `ppxf`, the smoothness of the output SFH is set by the `regul` keyword. This parameter essentially biases the solution towards that with the smoothest SFH which is still consistent with the data. As such, selecting the optimal `regul` value is essential. 

#### Remarks
---
* The regularisation treatment doesn't seem to work very well when a Gaussian kernel is used to construct the SFH. However, when the templates are light-weighted, the curves in the best-fit SFH appear quasi-Gaussian. 
* The optimum value of regul is highly sensitive to noise: it changes quite significantly for different realisations of the noise.
* Even when only a young stellar population is included, an old stellar population appears in the fit when the templates are weighted by mass. However, when weighted by light, these old stellar populations contribute only a small amount of light. 
* When S/N = 25 and a single, old stellar population is used, `regul` has a well-behaved minimum, but `ppxf` over-predicts the age of the stellar population (only a by a little bit) and predicts a much smoother SFH. Could this be because there is little evolution in the continuum at old ages? What happens if we instead try a single young stellar population?
* To define the smoothness of our input SFH, we use a Gaussian kernel that has a constant width in age space. Due to the logarithmic spacing of the templates in stellar age, this means that the young component is very broad, and the old component is very narrow, when the SFH is viewed in log age space. However, `ppxf` doesn't incorporate this into the regularisation process: the width of the smoothing kernel is constant in *log* age space. This effectively means that, at least using our templates, `ppxf` can't accurately fit the SFH we've provided it. **Possible workarounds**:
	* Define the SFH as having a constant width in log age space. Drawback: is this physical? If the galaxy has formed via multiple mergers from galaxies of a range of masses, then we might expect a spread of ages in the old stellar population. 
    * Define different SF durations for the young & old stellar populations: e.g. 1e9 yr for the old population, 1e7 for the young pop.
    
#### Ideas for quantifying the starburst timescale
---
* Time taken to form >50% (or some other percentage) of the mass associated with the most recent SF event
	* Could fit a Gaussian or some other polynomial function to get a more precise estimate 
* Read off the time corresponding to the peak in the SFH
	* again, could fit a Gaussian/polynomial to measure this 
* Mass-weighted mean age in all the stars <100 Myr old

In [1]:
%matplotlib widget

In [66]:
# imports 
import os
import numpy as np
from scipy.signal import convolve2d

from ppxftests.run_ppxf import run_ppxf
from ppxftests.mockspec import create_mock_spectrum, get_age_and_metallicity_values
from ppxftests.mockspec import N_AGES_PADOVA, N_MET_PADOVA, N_AGES_GENEVA, N_MET_GENEVA, FWHM_WIFES_INST_A

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

## TEST 1: sensitivity to the smoothness of the SFH
---

In [67]:
################################################################################################################
# Define the SFH
################################################################################################################
oversample_factor = 24

# SFH properties
age_1 = 10e9    # Age of primary stellar pop., in yr
met_1 = 0.006   # Metallicity of primary stellar pop., in solar units
mass_1 = 1e10   # Total stellar mass
sigma_age_1 = 5e8    # Duration of SF event (Gaussian sigma)
sigma_met_1 = 0.001  # Width in metallicity distribution (Gaussian sigma)

age_2 = 10e6    # Age of primary stellar pop., in yr
met_2 = 0.019   # Metallicity of primary stellar pop., in solar units
mass_2 = 1e7    # Total stellar mass
sigma_age_2 = 5e6    # Duration of SF event (Gaussian sigma)
sigma_met_2 = 0.001  # Width in metallicity distribution (Gaussian sigma)

sfh_components = [[age_1, met_1, sigma_age_1, sigma_met_1, mass_1], 
                  [age_2, met_2, sigma_age_2, sigma_met_2, mass_2]]

# Other parameters
SNR = 25
z = 0.05
sigma_star_kms = 250


In [68]:
################################################################################################################
# Compute the SFH
################################################################################################################
# Oversampled arrays in age and metallicity
ages, metallicities = get_age_and_metallicity_values(isochrones)
ages_oversampled = np.exp(np.linspace(np.log(ages[0]), 
                                      np.log(ages[-1]), 
                                      N_AGES_PADOVA * oversample_factor))
metallicities_oversampled = np.exp(np.linspace(np.log(metallicities[0]), 
                                      np.log(metallicities[-1]), 
                                      N_MET_PADOVA * oversample_factor))
xx, yy = np.meshgrid(ages_oversampled, metallicities_oversampled)
sfh_oversampled = np.zeros(xx.shape)

# Compute the SFH
fig, axs = plt.subplots(nrows=len(sfh_components) + 2, ncols=1, figsize=(8, 5 * (len(sfh_components) + 2)))
ii = 0
for age, met, sigma_age, sigma_met, mass in sfh_components:

    # Find the indices in the age & metallicity vectors corresponding to this age, metallicity
    age_idx = np.nanargmin(np.abs(ages_oversampled - age))
    met_idx = np.nanargmin(np.abs(metallicities_oversampled - met))

    # Define a "smooth" SFH
    sfh_component = np.exp(- (xx - age)**2 / (2 * sigma_age**2)) *\
                    np.exp(- (yy - met)**2 / (2 * sigma_met**2))
    sfh_component /= np.nansum(sfh_component)
    sfh_component *= mass
    
    # Add to the overall SFH
    sfh_oversampled += sfh_component
    
    # Plot
    bbox = axs[ii].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.03, bbox.height])
    m = axs[ii].imshow(sfh_component,
                       extent=(np.log10(ages[0]), np.log10(ages[-1]), metallicities[0], metallicities[-1]), 
                       aspect="auto")
    plt.colorbar(mappable=m, cax=cax)
    cax.set_ylabel(r"Mass ($M_\odot$)")
    axs[ii].set_title(f"SFH component {ii + 1}")
    ii += 1

# Plot to compare 
bbox = axs[ii].get_position()
cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.03, bbox.height])
m = axs[ii].imshow(sfh_oversampled,
                   extent=(np.log10(ages[0]), np.log10(ages[-1]), metallicities[0], metallicities[-1]), 
                   aspect="auto")
plt.colorbar(mappable=m, cax=cax)
cax.set_ylabel(r"Mass ($M_\odot$)")
axs[ii].set_title(f"Total SFH (oversampled)")
ii += 1

# Downsample & plot again
def bin2d(a, K):
    m_bins = a.shape[0] // K
    n_bins = a.shape[1] // K
    return a.reshape(m_bins, K, n_bins, K).sum(3).sum(1)

sfh = bin2d(sfh_oversampled, oversample_factor)
bbox = axs[ii].get_position()
cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.03, bbox.height])
m = axs[ii].imshow(np.log10(sfh),
                   extent=(np.log10(ages[0]), np.log10(ages[-1]), metallicities[0], metallicities[-1]), 
                   aspect="auto")
plt.colorbar(mappable=m, cax=cax)
cax.set_ylabel(r"Mass ($M_\odot$)")
axs[ii].set_title(f"Total SFH (downsampled)")

# 1. a very smooth SFH

# 2. a very spiky SFH

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Total SFH (downsampled)')

In [58]:
###########################################################################
# CREATE THE MOCK SPECTRUM
###########################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
###########################################################################
# RUN PPXF
###########################################################################
run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
         FWHM_inst_A=FWHM_WIFES_INST_A, 
         z=z, ngascomponents=1,
         auto_adjust_regul=True,
         isochrones="Padova", tie_balmer=True)

Median SNR = 24.9978
Emission lines included in gas templates:
['Balmer' '[OII]3726' '[OII]3729' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d' '[NeIII]3869' 'HeI3889']
Best Fit:       Vel     sigma
 comp. 0:     14628       254
 comp. 1:     14505       140
chi2/DOF: 1.072
method = capfit ; Jac calls: 5 ; Func calls: 52 ; Status: 2
Gas Reddening E(B-V): 0.000
Nonzero Templates:  11  /  224
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp: 1  Balmer (component 1)     0.2959      0.16   14505   140
Comp: 1  [OII]3726 (component 1)          0     0.091   14505   140
Comp: 1  [OII]3729 (component 1)          0     0.096   14505   140
Comp: 1  [OIII]5007_d (component 1)     0.1697      0.15   14505   140
Comp: 1  [OI]6300_d (component 1)          0      0.13   14505   140
Comp: 1  [NII]6583_d (component 1)     0.3872      0.12   14505   140
Comp: 1  [NeIII]3869 (comp

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/run_ppxf.py:398: RuntimeWarning: divide by zero encountered in log10
  m = ax_hist.imshow(np.log10(weights_mass_weighted), cmap="cubehelix_r", origin="lower", aspect="auto", vmin=0, vmax=np.nanmax(np.log10(weights_mass_weighted)))


Scaling noise by 1.0354...
Running ppxf on 20 threads...
Best Fit:       Vel     sigma
 comp. 0:     14628       254
 comp. 1:     14505       140
chi2/DOF: 1
method = capfit ; Jac calls: 5 ; Func calls: 52 ; Status: 2
Gas Reddening E(B-V): 0.000
Nonzero Templates:  11  /  224
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp: 1  Balmer (component 1)     0.2959      0.17   14505   140
Comp: 1  [OII]3726 (component 1)          0     0.094   14505   140
Comp: 1  [OII]3729 (component 1)          0       0.1   14505   140
Comp: 1  [OIII]5007_d (component 1)     0.1697      0.16   14505   140
Comp: 1  [OI]6300_d (component 1)          0      0.13   14505   140
Comp: 1  [NII]6583_d (component 1)     0.3872      0.12   14505   140
Comp: 1  [NeIII]3869 (component 1)    0.07112     0.093   14505   140
Comp: 1  HeI3889 (component 1)          0     0.075   14505   140
--

method = capfit ; Jac calls: 6 ; Func calls: 62 ; Status: 2
Gas Reddening E(B-V): 0.000
Nonzero Templates:  80  /  224
 comp. 0:     14628       253
 comp. 1:     14519        61
chi2/DOF: 1.002
method = capfit ; Jac calls: 6 ; Func calls: 62 ; Status: 2
Gas Reddening E(B-V): 0.000
Nonzero Templates:  75  /  224
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp: 1  Balmer (component 1)     0.1898      0.11   14519    60
Comp: 1  [OII]3726 (component 1)          0     0.056   14519    60
Best Fit:       Vel     sigma
 comp. 0:     14628       253
Comp: 1  [OII]3729 (component 1)          0     0.059   14519    60
Comp: 1  [OIII]5007_d (component 1)    0.09515      0.11   14519    60
Comp: 1  [OI]6300_d (component 1)          0     0.091   14519    60
Comp: 1  [NII]6583_d (component 1)     0.2674     0.085   14519    60
 comp. 1:     14519        60
chi2/DOF: 1.

...

--KeyboardInterrupt--
--KeyboardInterrupt--


Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 183, in cb


--KeyboardInterrupt--


KeyboardInterrupt: 


--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
ipdb> q
Exiting Debugger.


## TEST 2: sensitivity to young stellar populations 
---
Run `ppxf` on a series of templates with an young stellar population superimposed on an old stellar population. Vary the mass fraction of the young stellar population. 
Apply smoothing to the SFH to avoid regularisation issues.

In [ ]:
# Define the SFH. 